In [121]:
import sys
from pathlib import Path
import json

import pandas as pd
import gtfs_kit as gk
import itertools
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime

In [122]:
#setting GTFS data directory path for loading
DIR = Path('..')
sys.path.append(str(DIR))
DATA_DIR = DIR/'gtfs_data/' #GTFS datasets to be processed must be downloaded or stored here.

#loading GTFS data and list the feed components. You may edit the path variable depending upon where the GTFS dataset is stored.
path = DATA_DIR/'gtfs-openov-nl.zip'
gk.list_feed(path)

#Setting GTFS data details. REGION_NAME attribute is the region covered by the available GTFS dataset. For the openov dataset, it is the Netherlands.
REGION_NAME = 'netherlands'

#Set the calendar day(s) for generating the time schedule table in YYYYMMDD string format
CAL_DATE = '20200131'
date_text = 'Jan_sunday_20200131'

In [123]:
routes = pd.read_csv(DIR/'gtfs_data/{}/routes.txt'.format(REGION_NAME), low_memory=False)
routes = routes[['route_id', 'route_type']]
routes.head()

,route_id,route_type
0,67248,3
1,66859,3
2,66873,3
3,66872,3
4,67392,2


In [124]:
calendar_dates = pd.read_csv(DIR/'gtfs_data/{}/calendar_dates.txt'.format(REGION_NAME), low_memory=False)
calendar_dates.head()

,service_id,date,exception_type
0,1,20200120,1
1,2,20200120,1
2,2,20200210,1
3,3,20200120,1
4,3,20200203,1


In [125]:
#Subsetting services on the calendar date
services_of_interest = calendar_dates.query('date == {}'.format(CAL_DATE)).service_id.values
services_of_interest[0:5]

array([41, 67, 69, 73, 74], dtype=int64)

In [126]:
#Subsetting the trips for the date of interest
trips = pd.read_csv(DIR/'gtfs_data/{}/trips.txt'.format(REGION_NAME), low_memory=False)
trips = trips[['route_id', 'service_id', 'trip_id', 'shape_id', 'direction_id']]
trips = trips.loc[trips['service_id'].isin(services_of_interest)]
print(trips.head())

trips_of_interest = trips.trip_id.values
print(trips_of_interest[0:5])

    route_id  service_id    trip_id  shape_id  direction_id
17     17779         231   98650022  799314.0             1
18     41216         232  103436999  800108.0             0
19     62265         415  103577453  857372.0             1
20     62265         415  103577827  857432.0             0
21     62265         415  103583924  857372.0             1
[ 98650022 103436999 103577453 103577827 103583924]


In [127]:
stop_times = pd.read_csv(DIR/'gtfs_data/{}/stop_times.txt'.format(REGION_NAME), low_memory=False)
stop_times = stop_times.loc[stop_times['trip_id'].isin(trips_of_interest)]
stop_times['stop_id'] = stop_times['stop_id'].astype(str)

stop_times = stop_times[['trip_id', 'stop_sequence', 'stop_id', 'arrival_time', 'departure_time']]
for string in range(24, 50):
    stop_times = stop_times[~stop_times.arrival_time.str.startswith("{}:".format(string))]
stop_times.head()

,trip_id,stop_sequence,stop_id,arrival_time,departure_time
28,101870885,30,7195,14:12:00,14:12:00
29,101894943,39,650245,09:09:00,09:09:00
30,101894943,13,649922,08:48:00,08:48:00
31,101894943,7,649942,08:41:00,08:41:00
32,101909764,15,6855,15:16:00,15:16:00


In [128]:
stop_times = pd.merge(stop_times, trips, on="trip_id")
stop_times['arrival_time'] = pd.to_datetime(stop_times['arrival_time'],format= '%H:%M:%S' ).dt.time
stop_times.head()

,trip_id,stop_sequence,stop_id,arrival_time,departure_time,route_id,service_id,shape_id,direction_id
0,101870885,30,7195,14:12:00,14:12:00,41726,1659,838609.0,1
1,101870885,20,221202,14:00:00,14:00:00,41726,1659,838609.0,1
2,101870885,31,7200,14:13:00,14:13:00,41726,1659,838609.0,1
3,101870885,13,5950,13:50:00,13:50:00,41726,1659,838609.0,1
4,101870885,34,7276,14:15:00,14:15:00,41726,1659,838609.0,1


In [129]:
route_list = stop_times.route_id.unique()
direction_list = stop_times.direction_id.unique()

cat = {
    'route_id': route_list,
    'direction_id' : direction_list
}

order = cat.keys()
time_schedule = pd.DataFrame(itertools.product(*[cat[k] for k in order]), columns=order)
time_schedule['start_time'] = np.nan
time_schedule['stop_sequence'] = np.nan
time_schedule['inter_stop_tt'] = np.nan

time_schedule.head()

,route_id,direction_id,start_time,stop_sequence,inter_stop_tt
0,41726,1,NaN,NaN,NaN
1,41726,0,NaN,NaN,NaN
2,67002,1,NaN,NaN,NaN
3,67002,0,NaN,NaN,NaN
4,43157,1,NaN,NaN,NaN


In [130]:
#time_schedule = time_schedule.head(1)
for i in range(time_schedule.shape[0]):
    curr_route_stop_times = stop_times.query('route_id == {0} and direction_id == {1} and stop_sequence == 1'.format(time_schedule.route_id[i], time_schedule.direction_id[i]))
    curr_route_stop_times = curr_route_stop_times.reset_index()
    #print(curr_route_stop_times.head())
    
    if curr_route_stop_times.shape[0] > 0:
        start_times = curr_route_stop_times.arrival_time.sort_values().astype(str)
        start_array = [','.join(ele.split()) for ele in start_times]
        time_schedule.start_time[i] = start_array
        #print(i)

        #adding time intervals between stops
        first_trip_in_curr_route = stop_times.query('trip_id == {}'.format(curr_route_stop_times.trip_id[0]))
        first_trip_in_curr_route = first_trip_in_curr_route.sort_values('stop_sequence')
        #print(first_trip_in_curr_route.head())

        travel_times = pd.to_timedelta(first_trip_in_curr_route['arrival_time'].astype(str)).diff(-1).dt.total_seconds()
        #print(travel_times)
        tt_array = [','.join(ele.split()) for ele in abs(travel_times[:-1]).astype(int).astype(str)]
        time_schedule.inter_stop_tt[i] = tt_array

        stop_sequence = first_trip_in_curr_route.stop_id.astype(str)
        stop_array = [','.join(ele.split()) for ele in stop_sequence]
        time_schedule.stop_sequence[i] = stop_array
    print('route {} in direction {} computed.'.format(time_schedule.route_id[i],time_schedule.direction_id[i]))
    
print(time_schedule)
        

route 41726 in direction 1 computed.
route 41726 in direction 0 computed.
route 67002 in direction 1 computed.
route 67002 in direction 0 computed.
route 43157 in direction 1 computed.
route 43157 in direction 0 computed.
route 200 in direction 1 computed.
route 200 in direction 0 computed.
route 206 in direction 1 computed.
route 206 in direction 0 computed.
route 41734 in direction 1 computed.
route 41734 in direction 0 computed.
route 53530 in direction 1 computed.
route 53530 in direction 0 computed.
route 41866 in direction 1 computed.
route 41866 in direction 0 computed.
route 67083 in direction 1 computed.
route 67083 in direction 0 computed.
route 67079 in direction 1 computed.
route 67079 in direction 0 computed.
route 63410 in direction 1 computed.
route 63410 in direction 0 computed.
route 57594 in direction 1 computed.
route 57594 in direction 0 computed.
route 60897 in direction 1 computed.
route 60897 in direction 0 computed.
route 67150 in direction 1 computed.
route 671

In [131]:
time_schedule = pd.merge(time_schedule, routes, on="route_id")
time_schedule.head(10)

,route_id,direction_id,start_time,stop_sequence,inter_stop_tt,route_type
0,41726,1,"[05:49:00, 05:55:00, 06:46:00, 06:54:00, 07:23...","[1479722, 343140, 6639, 480533, 5750, 1480119,...","[120, 60, 120, 120, 360, 240, 60, 120, 60, 180...",3
1,41726,0,"[06:18:00, 06:31:00, 07:01:00, 07:31:00, 08:05...","[221260, 5520, 1480064, 539293, 6816, 7146, 74...","[120, 120, 60, 60, 60, 180, 60, 60, 60, 180, 1...",3
2,67002,1,"[05:58:00, 06:15:00, 07:15:00, 08:17:00, 09:17...","[1538515, 1525504, 1240604, 1240569, 650428, 1...","[60, 60, 60, 60, 180, 60, 60, 120, 120, 120, 6...",3
3,67002,0,"[05:35:00, 06:35:00, 07:35:00, 08:35:00, 09:35...","[649655, 649451, 650033, 649996, 649942, 64982...","[120, 120, 60, 60, 60, 60, 120, 60, 120, 120, ...",3
4,43157,1,NaN,NaN,NaN,3
5,43157,0,"[09:25:00, 09:45:00, 10:05:00, 10:25:00, 10:45...","[649584, 5912, 6496, 157878, 1927926, 7380, 57...","[180, 60, 0, 60, 0, 60, 0, 60, 0, 60, 60, 60, ...",3
6,200,1,"[06:13:00, 06:43:00, 07:13:00, 07:43:00, 08:13...","[5703, 7286, 109781, 5516, 6843, 6252, 1546702...","[0, 60, 0, 60, 60, 60, 60, 0, 60, 60, 60, 60, ...",3
7,200,0,"[06:00:00, 06:30:00, 07:00:00, 07:30:00, 08:00...","[1961657, 6924, 6738, 5791, 1501652, 5929, 664...","[120, 60, 0, 60, 60, 0, 60, 60, 0, 120, 60, 60...",3
8,206,1,"[06:16:00, 06:37:00, 06:56:00, 07:16:00, 07:37...","[548287, 5872, 6206, 109785, 63642, 5511, 5625...","[60, 0, 60, 60, 0, 60, 60, 0, 60, 120, 60, 0, ...",3
9,206,0,"[06:00:00, 06:21:00, 06:40:00, 07:00:00, 07:21...","[827054, 5975, 1479912, 7070, 1533365, 1927900...","[180, 60, 120, 60, 60, 60, 60, 60, 0, 60, 60, ...",3


In [132]:
print(time_schedule.route_type.unique())
print(time_schedule.shape[0])

#0 - Tram, Streetcar, Light rail. Any light rail or street level system within a metropolitan area.
#1 - Subway, Metro. Any underground rail system within a metropolitan area.
#2 - Rail. Used for intercity or long-distance travel.
#3 - Bus. Used for short- and long-distance bus routes.
#4 - Ferry. Used for short- and long-distance boat service.

# More details at https://developers.google.com/transit/gtfs/reference#tripstxt

[3 0 1 2 4]
4892


In [136]:
folder = '../gtfs_data/netherlands/time_schedules/'
time_schedule.to_csv('{0}{1}.csv'.format(folder,date_text), header=True, index=False) 